# OvP project

## Imports and data loading

In [1]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import yake

from keybert import KeyBERT
from langdetect import detect

import openai
from keybert.llm import OpenAI
from keybert import KeyLLM


C:\Users\sue-e\anaconda3\envs\Thesis2\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# import data
data = pd.read_excel("raw_data.xlsx")

print(data.head(2))

        CURSUS             LANGE_NAAM_NL  \
0   ASB-ACT-EN  ACT - Leef zoals je wilt   
1  ASB-ACTLEEF  ACT - Leef zoals je wilt   

                                          DOCENT_ROL  \
0  Brasjen DOCENT; Bruinsma DOCENT; Buiks DOCENT;...   
1  Brasjen DOCENT; Bruinsma DOCENT; Buiks DOCENT;...   

  OSS_ADF_UTILITY.HTML_TO_TEXT(H.INHOUD)  
0                                    NaN  
1                                    NaN  


### Exploring data

In [4]:

print(data.columns)
teacher_list_raw = data['DOCENT_ROL'].tolist()
print(teacher_list_raw[:5])


Index(['CURSUS', 'LANGE_NAAM_NL', 'DOCENT_ROL',
       'OSS_ADF_UTILITY.HTML_TO_TEXT(H.INHOUD)'],
      dtype='object')
['Brasjen DOCENT; Bruinsma DOCENT; Buiks DOCENT; Burgh DOCENT; Galipò DOCENT; Gronden DOCENT; Kaathoven CONTACTPERSOON; Verhagen DOCENT', 'Brasjen DOCENT; Bruinsma DOCENT; Buiks DOCENT; Burgh DOCENT; Galipò DOCENT; Gronden DOCENT; Kaathoven CONTACTPERSOON; Verhagen DOCENT', 'Bertens DOCENT; Brasjen DOCENT; Galipò DOCENT', 'Baars DOCENT; Brasjen DOCENT; Breemen DOCENT; Bruinsma DOCENT; Buiks DOCENT; Burgh DOCENT; Galipò DOCENT; Gronden DOCENT; Kaathoven CONTACTPERSOON; Schrijen DOCENT; Spreeuwenberg DOCENT; Verhagen DOCENT; Winters DOCENT', 'Baars DOCENT; Brasjen DOCENT; Breemen DOCENT; Bruinsma DOCENT; Buiks DOCENT; Burgh DOCENT; Galipò DOCENT; Gronden DOCENT; Schrijen DOCENT; Spanbroek CONTACTPERSOON; Spreeuwenberg DOCENT; Verhagen DOCENT']


In [5]:
teacher_list = [
    item.strip()
    for entry in teacher_list_raw
    for item in entry.split(';')
]
print(teacher_list[:10])

['Brasjen DOCENT', 'Bruinsma DOCENT', 'Buiks DOCENT', 'Burgh DOCENT', 'Galipò DOCENT', 'Gronden DOCENT', 'Kaathoven CONTACTPERSOON', 'Verhagen DOCENT', 'Brasjen DOCENT', 'Bruinsma DOCENT']


### Choose keyword model by example

In [6]:
# yake function
def extract_keywords(text):
    kw_extractor = yake.KeywordExtractor(
                            lan="en",           # or appropriate language code
                            n=3,                # max n-gram size (try 2-3)
                            dedupLim=0.8,       # make deduplication stricter
                            dedupFunc='levs',   # try 'levs' or 'jaro' instead of default 'seqm'
                            windowsSize=2,      # context window size
                            top=15              # how many keywords you want
                        )
    keywords = kw_extractor.extract_keywords(text)
    return [kw for kw, score in keywords]

In [13]:
example_text = "Deze cursus gaat over de visuele cultuur van de Nederlandse zeventiende-eeuwse Republiek. De ongekende commerciële groei van de Verenigde Nederlanden kwam voort uit expansiedrang en kolonialisme, wat onder meer resulteerde in een bloeiende kunst- en rariteitenhandel en een explosie van artistieke creativiteit. Deze cursus onderzoekt de impact van de steeds groter wordende wereld op de beeldende kunsten. Welke rol speelden kunstwerken en andere exotische kunstobjecten die in de huiselijke sfeer werden verzameld en gekoesterd? Wat werd er afgebeeld, waarom en voor wie? Wat onthullen schilderijen over de houding van de Nederlanders ten opzichte van de koloniën, van andere Europese landen en van zichzelf? Wat vertellen rariteitenverzamelingen ons over de verhouding tussen het zelf en de wereld, het vertrouwde en het vreemde, het lokale en het internationale? Maar ook zal er stil worden gestaan bij ontdekkingen die op wetenschappelijk gebied worden gedaan en hoe deze al vlug hun weg weten te vinden in de beeldende kunst. [NB: de cursusinhoud is onder voorbehoud en kan wijzigen]"


lang = detect(example_text)
stop_words = "dutch" if lang.lower() == "nl" else "english"

kw_model = KeyBERT()
keywords = kw_model.extract_keywords(
                                example_text,
                                keyphrase_ngram_range=(1,3),
                                stop_words=stop_words,
                                # use_maxsum=True,
                                # nr_candidates=20,
                                top_n=20,
                                # use_mmr=True,
                                # diversity=0.7
)

print(f"Language: {lang} → Stopwords: {stop_words}")
print(keywords)
print()

Language: nl → Stopwords: dutch
[]


## Get keywords per topic


In [8]:
course_dict = {}

for ind, row in data.iterrows():
    # print(row)
    # print("done: ",row[0], row[1], row[2])
    course_code, course_name, teachers, keywords = row
    if pd.isna(keywords):
        continue
        
    teachers = teachers.split(";")
    teachers = list(set([' '.join(item.split()[:-1]) for item in teachers]))
    
    # using yake, keywords not great
    keywords_yake = extract_keywords(keywords)
    
    # use keybert, slightly better but not perfect yet
    kw_model = KeyBERT()
    keywords_bert = kw_model.extract_keywords(keywords,
                                              stop_words='english',
                                              use_maxsum=True,
                                              nr_candidates=20,
                                              top_n=20, 
                                              keyphrase_ngram_range=(1, 3),
                                              use_mmr=True,
                                              diversity=0.7)
    # print(keywords)
    
    course_dict[row[0]] = {"course_code": course_code,
                           "course_name": course_name,
                           "teachers": teachers,
                           "keywords_yake": keywords_yake,
                           "keywords_bert": keywords_bert}
    break
    

print("--------------------------------")
print(course_dict)
    # if ind > 3:
    #     break

--------------------------------
{'FTR-CC300': {'course_code': 'FTR-CC300', 'course_name': 'Persoonlijke en professionele ontwikkeling 3: Geesteswetenschappen in de praktijk', 'teachers': ['Rietdijk', 'Linssen'], 'keywords_yake': ['van', 'het', 'het teken van', 'Deze cursus staat', 'wat', 'Deze cursus', 'cursus', 'teken van', 'gaat het', 'zoektocht', 'het teken', 'een', 'cursus staat', 'eigen professionele houding', 'ten behoeve van'], 'keywords_bert': [('drijfveren waarden kwaliteiten', 0.5478), ('theologische religiewetenschappelijke traditie', 0.4302), ('wat je nog', 0.394), ('eigen professionele', 0.3407), ('eerste positie', 0.3265), ('zoektocht', 0.3252), ('bedoeld voor ftr', 0.2872), ('cursus speciaal', 0.2521), ('ene kant en', 0.2412), ('tevens', 0.2192), ('filosofie theologie', 0.2183), ('samenleving deze', 0.1996), ('elkaar', 0.1676), ('publiek uitmondend', 0.1495), ('debat', 0.1463), ('door jou', 0.14), ('invulling', 0.1343), ('analyse uit heden', 0.0925), ('ambities', 0.0559

### Try using LLM API